In [100]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashlineups
import cv2
import matplotlib.image as image
from IPython.core.display import HTML

fig_DIR = "../figs/teams/"
box_DIR = "../data/box/"
img_DIR_T = "../data/images/teams/"
img_DIR_P = "../data/images/players/"

## 5 Man Lineups

In [139]:
stats = leaguedashteamstats.LeagueDashTeamStats(
    measure_type_detailed_defense="Advanced",
)
df1 = stats.get_data_frames()[0]
df1 = df1.sort_values("W_PCT",ascending=False).reset_index(drop=True)
df1["OPTS"] = df1["OFF_RATING"]*df1["POSS"]
df1["DPTS"] = df1["DEF_RATING"]*df1["POSS"]
lavg_Rtg = round(df1["OPTS"].sum()/df1["POSS"].sum(),2)

In [140]:
stats = leaguedashlineups.LeagueDashLineups(
    group_quantity=5,
    measure_type_detailed_defense="Advanced",
    per_mode_detailed="Totals",
    pace_adjust="Y",
    plus_minus="Y",
    rank="Y",
    season_type_all_star="Regular Season"
)
df = stats.get_data_frames()[0]

In [126]:
df1["TEAM_ABBREVIATION"].unique()

array(['NYK', 'HOU', 'SAC', 'MIN', 'LAC', 'BOS', 'TOR', 'POR', 'ATL',
       'DET', 'ORL', 'CHI', 'MIL', 'DEN', 'BKN', 'LAL', 'MIA', 'IND',
       'CLE', 'SAS', 'WAS', 'PHX', 'DAL', 'GSW', 'OKC', 'CHA', 'UTA',
       'PHI', 'NOP', 'MEM'], dtype=object)

In [135]:
df1 = df.sort_values("MIN",ascending=False).reset_index(drop=True)
df1["OFF_RATING"] = (df1["POSS"]*df1["OFF_RATING"]+ lavg_Rtg*550)/(df1["POSS"]+550)
df1["DEF_RATING"] = (df1["POSS"]*df1["DEF_RATING"]+ lavg_Rtg*850)/(df1["POSS"]+850)
df1["NET_RATING"] = df1["OFF_RATING"] - df1["DEF_RATING"]
df1["OFF_RATING"] = df1["OFF_RATING"].round(1)
df1["DEF_RATING"] = df1["DEF_RATING"].round(1)
df1["NET_RATING"] = df1["NET_RATING"].round(1)
# df2 = df1.nlargest(100,"MIN")
# df2 = df1.query("MIN >= 50")
df2 = df1.copy()
team = 'LAC'
df2 = df1.query(f"TEAM_ABBREVIATION == '{team}' & MIN >= 10")
min_min = df2["MIN"].min()
max_min = df2["MIN"].max()
len(df2)

20

In [136]:
var = "NET_RATING"
# var = "OFF_RATING"
# var = "DEF_RATING"
# var = "MIN"
df3 = df2.sort_values(var,ascending=False).reset_index(drop=True)
# df3 = df2.sort_values(var,ascending=True).reset_index(drop=True)
df3.index +=1
df3[["B1","P11","P21","P31","P41","P51","B2"]] = df3["GROUP_ID"].str.split("-",expand=True)
df3["P1"]=img_DIR_P + df3["P11"] + ".png"
df3["P2"]=img_DIR_P + df3["P21"] + ".png"
df3["P3"]=img_DIR_P + df3["P31"] + ".png"
df3["P4"]=img_DIR_P + df3["P41"] + ".png"
df3["P5"]=img_DIR_P + df3["P51"] + ".png"
df3["NET_PER"] = df3["NET_RATING"].rank(pct=True)*100
df3["OFF_PER"] = df3["OFF_RATING"].rank(pct=True)*100
df3["DEF_PER"] = df3["DEF_RATING"].rank(pct=True,ascending=False)*100

In [137]:
df4 = df3[["GROUP_NAME","P1","P2","P3","P4","P5",var,"MIN","NET_PER","OFF_PER","DEF_PER"]].head(10)
# df4 = df3[["GROUP_NAME","P1","P2","P3","P4","P5","MIN","NET_RATING","NET_PER","OFF_PER","DEF_PER"]].head(20)
df4["P1"] = df4["P1"].apply(path_to_image_html)
df4["P2"] = df4["P2"].apply(path_to_image_html)
df4["P3"] = df4["P3"].apply(path_to_image_html)
df4["P4"] = df4["P4"].apply(path_to_image_html)
df4["P5"] = df4["P5"].apply(path_to_image_html)
# df4 = df4.rename(columns={"GROUP_NAME":f"Best {var}"})
df4 = df4.style.set_caption(f'Best {var}').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'red'),
        ('font-size', '25px'),
        ('font-style', 'italic'),
        ('font-weight', 'bold'),
        ('text-align', 'center')
    ]
}])
df4 = df4.background_gradient(axis=0,subset=["NET_PER","OFF_PER","DEF_PER"], vmin=0, vmax=100, cmap="RdYlGn")
df4 = df4.background_gradient(axis=0,subset=["MIN"], vmin=min_min, vmax=max_min, cmap="RdYlGn")
# df4 = df4.background_gradient(axis=0,subset=["NET_RATING"], vmin=df2["NET_RATING"].min(), vmax=df2["NET_RATING"].max(), cmap="RdYlGn")
df4 = df4.format(formatter={'MIN':'{:,.0f}',var:'{:,.1f}',"NET_PER":'{:,.1f}',"OFF_PER":'{:,.1f}',"DEF_PER":'{:,.1f}'})
# df4 = df4.format(formatter={'MIN':'{:,.0f}',"NET_PER":'{:,.1f}',"OFF_PER":'{:,.1f}',"DEF_PER":'{:,.1f}',"NET_RATING":'{:,.1f}'})
HTML(df4.to_html(escape=False))

,GROUP_NAME,P1,P2,P3,P4,P5,NET_RATING,MIN,NET_PER,OFF_PER,DEF_PER
1,J. Harden - K. Dunn - I. Zubac - D. Jones Jr. - A. Coffey,,,,,,3.7,84,100.0,100.0,100.0
2,N. Batum - J. Harden - K. Dunn - A. Coffey - K. Porter Jr.,,,,,,2.5,12,95.0,90.0,90.0
3,J. Harden - N. Powell - K. Dunn - I. Zubac - D. Jones Jr.,,,,,,2.3,84,90.0,95.0,80.0
4,N. Batum - N. Powell - K. Dunn - A. Coffey - K. Porter Jr.,,,,,,1.8,13,85.0,82.5,70.0
5,N. Batum - I. Zubac - A. Coffey - T. Mann - K. Porter Jr.,,,,,,1.5,12,80.0,70.0,85.0
6,N. Batum - M. Bamba - T. Mann - K. Porter Jr. - J. Miller,,,,,,1.0,21,75.0,60.0,75.0
7,N. Batum - K. Dunn - I. Zubac - A. Coffey - K. Porter Jr.,,,,,,0.9,54,70.0,40.0,95.0
8,J. Harden - N. Powell - K. Dunn - I. Zubac - T. Mann,,,,,,0.7,11,62.5,55.0,62.5
9,J. Harden - N. Powell - K. Dunn - I. Zubac - A. Coffey,,,,,,0.7,10,62.5,75.0,40.0
10,J. Harden - N. Powell - D. Jones Jr. - T. Mann - K. Jones,,,,,,0.6,11,55.0,82.5,30.0


In [138]:
asdsad

NameError: name 'asdsad' is not defined

In [7]:
df1g = df1.groupby(["TEAM_ABBREVIATION","TEAM_ID"])
keys = list(df1g.groups)

In [8]:
teams_dict,_ = get_teams()

In [9]:
dfa = []
for key in keys:
    df3 = df1g.get_group(key)
    df4 = df3.iloc[:5]
    df4["Pos"] = df4["NET_RATING"]>0
    df5 = pd.DataFrame({"TeamID":key[1],"Team":key[0],"Pos":[df4["Pos"].sum()]})
    dfa.append(df5)
df6 = pd.concat(dfa)
df6["nameTeam"]=df6["TeamID"].map(teams_dict)
df6 = df6.sort_values("Pos",ascending=False).reset_index(drop=True)
df_teams = pd.read_csv("../data/NBA_teams_colors_logos.csv")
df6 = pd.merge(df6,df_teams,on="nameTeam")
teams = df6["Team"].to_list()
# teams.reverse()
df6["Team"] = pd.Categorical(df6["Team"],categories=teams)
df6["image"]= img_DIR_T + df6["nameTeam"] + ".png"


In [10]:
p = (
    ggplot(df6,aes(x="Team",y="Pos",image="image"))
    + geom_bar(aes(fill = "colorsTeam"),stat="identity", alpha=0.8)
    + geom_image(y=-0.1,size=0.07)
    + scale_color_identity(aesthetics = ["fill"])
    + scale_y_continuous(breaks=range(1,11,2)) 
    # + coord_flip()
    + theme_xkcd()
    # + theme_xkcd(stroke_color="black")
    + theme(
        figure_size=(10,8),
        plot_title=element_text(size=24),
        axis_title_x=element_blank(),
        axis_title_y=element_text(size=18),
        plot_caption=element_text(size=14),
        # text=element_text(color="white"),
        # panel_border=element_line(color="white"),
        # plot_background=element_rect(fill="dimgrey"),
        # panel_background=element_rect(fill="dimgrey")
    )
    + labs(
        title="Number of Positive Lineups with Top 5 in Mins Played",
        caption="@SravanNBA | source:nba.com/stats",
        y="# of Positive Lineups"
    )
)
p

<Figure Size: (1000 x 800)>

## 2 Man Lineups

In [141]:
stats = leaguedashlineups.LeagueDashLineups(
    group_quantity=2,
    measure_type_detailed_defense="Advanced",
    per_mode_detailed="Totals",
    pace_adjust="Y",
    plus_minus="Y",
    rank="Y",
    season_type_all_star="Regular Season"
)
df = stats.get_data_frames()[0]

In [143]:
df1 = df.sort_values("MIN",ascending=False).reset_index(drop=True)
# df2 = df1.nlargest(1000,"MIN")
df2 = df1.query("MIN >= 300")
len(df2)

162

In [58]:
var = "NET_RATING"
df3 = df2.sort_values(var,ascending=True).reset_index(drop=True)
# var = "DEF_RATING"
# df3 = df2.sort_values(var,ascending=True).reset_index(drop=True)
df3.index +=1
df3[["B1","P11","P21","B2"]] = df3["GROUP_ID"].str.split("-",expand=True)
df3["P1"]=img_DIR_P + df3["P11"] + ".png"
df3["P2"]=img_DIR_P + df3["P21"] + ".png"
df3["NET_PER"] = df3["NET_RATING"].rank(pct=True)*100
df3["OFF_PER"] = df3["OFF_RATING"].rank(pct=True)*100
df3["DEF_PER"] = df3["DEF_RATING"].rank(pct=True,ascending=False)*100

In [59]:
df4 = df3[["GROUP_NAME","P1","P2",var,"MIN","NET_PER","OFF_PER","DEF_PER"]].head(10)
# df4 = df3[["GROUP_NAME","P1","P2",var,"NET_PER","OFF_PER","DEF_PER"]].head(10)
df4["P1"] = df4["P1"].apply(path_to_image_html)
df4["P2"] = df4["P2"].apply(path_to_image_html)
# df4 = df4.rename(columns={"GROUP_NAME":f"Best {var}"})
df4 = df4.style.set_caption(f'Worst {var}').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'red'),
        ('font-size', '25px'),
        ('font-style', 'italic'),
        ('font-weight', 'bold'),
        ('text-align', 'center')
    ]
}])
df4 = df4.format(formatter={'MIN':'{:,.0f}',var:'{:,.1f}',"NET_PER":'{:,.1f}',"OFF_PER":'{:,.1f}',"DEF_PER":'{:,.1f}'})
# df4 = df4.format(formatter={var:'{:,.0f}',"NET_PER":'{:,.1f}',"OFF_PER":'{:,.1f}',"DEF_PER":'{:,.1f}'})

In [60]:
# HTML(df4.to_html(escape=False,formatters=dict(P1=path_to_image_html,P2=path_to_image_html)))
HTML(df4.to_html(escape=False))

,GROUP_NAME,P1,P2,NET_RATING,MIN,NET_PER,OFF_PER,DEF_PER
1,K. Kuzma - A. Sarr,,,-25.6,202,0.3,0.3,29.2
2,K. Kuzma - J. Poole,,,-23.5,209,0.6,1.4,10.6
3,J. Poole - A. Sarr,,,-23.3,262,0.8,2.0,6.3
4,J. Poole - C. Carrington,,,-22.6,225,1.1,1.7,14.8
5,R. Westbrook - J. Strawther,,,-22.1,215,1.4,0.8,29.9
6,K. Kuzma - B. Coulibaly,,,-22.0,200,1.7,1.1,22.8
7,A. Drummond - K. Oubre Jr.,,,-21.3,241,2.0,2.3,11.5
8,B. Coulibaly - C. Carrington,,,-20.6,306,2.3,5.1,3.7
9,B. Coulibaly - A. Sarr,,,-20.0,317,2.5,3.8,9.7
10,A. Dosunmu - J. Giddey,,,-19.9,242,2.8,25.4,0.3


In [61]:
dfs

NameError: name 'dfs' is not defined

## 3 Man Lineups

In [ ]:
stats = leaguedashlineups.LeagueDashLineups(
    group_quantity=3,
    measure_type_detailed_defense="Advanced",
    per_mode_detailed="Totals",
    pace_adjust="Y",
    plus_minus="Y",
    rank="Y",
    season_type_all_star="Regular Season"
)
df = stats.get_data_frames()[0]

In [ ]:
df1 = df.sort_values("MIN",ascending=False).reset_index(drop=True)
# df2 = df1.nlargest(1000,"MIN")
df2 = df1.query("MIN >= 500")
len(df2)

In [ ]:
# var = "OFF_RATING"
# df3 = df2.sort_values(var,ascending=False).reset_index(drop=True)
var = "DEF_RATING"
df3 = df2.sort_values(var,ascending=True).reset_index(drop=True)
df3.index +=1
df3[["B1","P11","P21","P31","B2"]] = df3["GROUP_ID"].str.split("-",expand=True)
df3["P1"]=img_DIR_P + df3["P11"] + ".png"
df3["P2"]=img_DIR_P + df3["P21"] + ".png"
df3["P3"]=img_DIR_P + df3["P31"] + ".png"
df3["NET_RATING_PER"] = df3["NET_RATING"].rank(pct=True)*100
df3["OFF_RATING_PER"] = df3["OFF_RATING"].rank(pct=True)*100
df3["DEF_RATING_PER"] = df3["DEF_RATING"].rank(pct=True,ascending=False)*100

In [ ]:
df4 = df3[["GROUP_NAME","P1","P2","P3",var,"MIN","NET_RATING_PER","OFF_RATING_PER","DEF_RATING_PER"]].head(10)
df4["P1"] = df4["P1"].apply(path_to_image_html)
df4["P2"] = df4["P2"].apply(path_to_image_html)
df4["P3"] = df4["P3"].apply(path_to_image_html)
# df4 = df4.rename(columns={"GROUP_NAME":f"Best {var}"})
df4 = df4.style.set_caption(f'Best {var}').set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'red'),
        ('font-size', '25px'),
        ('font-style', 'italic'),
        ('font-weight', 'bold'),
        ('text-align', 'center')
    ]
}])
df4 = df4.format(formatter={'MIN':'{:,.0f}',var:'{:,.1f}',"NET_RATING_PER":'{:,.1f}',"OFF_RATING_PER":'{:,.1f}',"DEF_RATING_PER":'{:,.1f}'})

In [ ]:
HTML(df4.to_html(escape=False))